In [1]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
import csv
from urllib.parse import urlparse

In [2]:
newspaper_feeds = {}
newspaper_feeds['websites'] = {}

#JSON File mapping news source to RSS Feed URL 
with open('News_RSS.json') as news_sources:
    newspaper_feeds = json.load(news_sources)
print(newspaper_feeds)

{'cnn_top_stories': 'http://rss.cnn.com/rss/cnn_topstories.rss', 'cnn_us': 'http://rss.cnn.com/rss/cnn_us.rss', 'cnn_politics': 'http://rss.cnn.com/rss/cnn_allpolitics.rss', 'cnn_health': 'http://rss.cnn.com/rss/cnn_health.rss', 'nyt_world': 'https://rss.nytimes.com/services/xml/rss/nyt/World.xml', 'nyt_us': 'https://rss.nytimes.com/services/xml/rss/nyt/US.xml', 'nyt_politics': 'https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml', 'nyt_health': 'https://www.nytimes.com/services/xml/rss/nyt/Health.xml', 'wsj': 'https://feeds.a.dj.com/rss/RSSWorldNews.xml', 'wsj_bus': 'https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml', 'npr': 'https://rss.app/feeds/KDeAWmBPh2mtUe2I.xml', 'npr_pol': 'https://rss.app/feeds/Sc0UB1rFfCi0bi5b.xml', 'npr_world': 'https://rss.app/feeds/1lZtUxLIEuxYKfUg.xml', 'wash_pol': 'https://rss.app/feeds/Mm636f8kUVcvhYrR.xml', 'wash_world': 'https://rss.app/feeds/8mMUCa6porKSpinr.xml', 'wash_health': 'https://rss.app/feeds/b4zIJKlL1ldrYDyr.xml', 'bbc': 'https://rss.a

In [3]:
def is_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

In [4]:
f = open("Real_Scrapped.csv","w")
writer = csv.writer(f)
writer.writerow(["url", "title", "text"])
count = 0
writes = 0
for key in newspaper_feeds:
    rss_feed = newspaper_feeds[key]
    rss_feed = fp.parse(rss_feed) #FP Parse to get feed entries as a list
    #each entry is a feedparser.FeedParserDict with 
    #['title', 'title_detail', 'summary', 'summary_detail', 'links', 'link', 'id', 'guidislink', 'published', 'published_parsed', 'media_content', 'feedburner_origlink']
    
   
    count += len(rss_feed['entries'])
    for i in rss_feed['entries']: 
        try:
            if is_url(i['id']):
                og_url = i['id'] #url to article, not rss url
            else:
                og_url = i['link']
            rss_url = i['link']
            if "video" not in og_url: #only what text articles
                article = Article(rss_url)
                article.download()
                article.parse()
                title = article.title
                text = article.text
                writer.writerow([og_url, '{}'.format(title), '{}'.format(text)])
                writes += 1
        except Exception as e:
            print(e)
            continue
        
f.close()
print(count)
print(writes)

Article `download()` failed with ('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing data: incorrect header check')) on URL http://rss.cnn.com/~r/rss/cnn_us/~3/Vm3oMwb3FOo/century-old-grocery-store-uses-modern-twist-to-deliver-goods-amid-pandemic
Article `download()` failed with 403 Client Error: Forbidden for url: https://wlos.com/news/local/local-shop-donates-time-and-material-to-make-masks-for-essential-workers?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+rss%2Fcnn_us+%28RSS%3A+CNN+-+U.S.%29 on URL http://rss.cnn.com/~r/rss/cnn_us/~3/eVLwONsTZjM/local-shop-donates-time-and-material-to-make-masks-for-essential-workers
Article `download()` failed with 403 Client Error: Forbidden for url: https://foxbaltimore.com/news/coronavirus/coronavirus-outbreak-reported-at-long-term-care-facility-in-carroll-county?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+rss%2Fcnn_us+%28RSS%3A+CNN+-+U.S.%29 on URL http://r

In [5]:
import pandas as pd
df = pd.read_csv("Real_Scrapped.csv")
print(df.size)
print(df.head())

2727
                                                 url  \
0  https://www.cnn.com/2020/03/29/politics/trump-...   
1  https://www.cnn.com/world/live-news/coronaviru...   
2  https://www.cnn.com/2020/03/29/politics/trump-...   
3  https://www.cnn.com/interactive/2020/health/co...   
4  https://www.cnn.com/2020/03/29/opinions/donald...   

                                               title  \
0  Trump extends federal social distancing guidel...   
1  Coronavirus live updates: Global pandemic kill...   
2  Trump concedes US coronavirus death toll could...   
3                  Tracking Covid-19 cases in the US   
4                         Donald Trump's special day   

                                                text  
0  Washington (CNN) President Donald Trump said S...  
1  The exterior of Elmhurst Hospital in Queens on...  
2  (CNN) President Donald Trump acknowledged Sund...  
3                                                NaN  
4  Sign up to get our new weekly column as a n

In [6]:
for i in df["url"]:
    print(i)

https://www.cnn.com/2020/03/29/politics/trump-coronavirus-press-conference/index.html
https://www.cnn.com/world/live-news/coronavirus-outbreak-03-29-20-intl-hnk/index.html
https://www.cnn.com/2020/03/29/politics/trump-deaths-coronavirus/index.html
https://www.cnn.com/interactive/2020/health/coronavirus-us-maps-and-cases/
https://www.cnn.com/2020/03/29/opinions/donald-trumps-special-day-opinion-weekly-column-galant/index.html
https://www.cnn.com/2020/03/29/politics/us-election-2020-week-ahead/index.html
https://www.cnn.com/2020/03/29/politics/fact-check-coronavirus-briefing/index.html
https://www.cnn.com/2020/03/29/us/new-york-coronavirus-cases/index.html
https://www.cnn.com/2020/03/29/politics/fauci-trump-quarantine-white-house-discussions/index.html
https://www.cnn.com/2020/03/29/us/maria-mercader-death-trnd/index.html
https://www.cnn.com/2020/03/29/opinions/coronavirus-personal-protective-equipment-obeidallah/index.html
https://www.cnn.com/2020/03/29/politics/sotu-jake-tapper-pelosi-

In [7]:
df.shape

(909, 3)